In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import sqlite3

## Database connection

In [ ]:
con = sqlite3.connect(r'../input/amazon-customers-data/database.sqlite')
cur = con.cursor()

cur.execute('''
            SELECT *
            FROM Reviews
            LIMIT 1000; ''')

## Create a dataframe

In [ ]:
colunas = [i[0] for i in cur.description]
df = pd.DataFrame(cur.fetchall(), columns=colunas)

In [ ]:
print(df.shape)
df.head(5)

## Exploratory Analysis

In [ ]:
px.histogram(df['Score'], title='Score Distribution')

### Sentiment Analysis

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install pip install spacytextblob

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

In [ ]:
sentimento = []
for n in df['Text']:
    doc = nlp(n)
    sentimento.append(doc._.polarity)

In [ ]:
sentimento = pd.DataFrame(sentimento, columns=['Sentimento'])
df2 = pd.concat([df,sentimento], axis=1)

In [ ]:
humor = []
for n in df2['Sentimento']:
    if n >= 0.5:
        humor.append("Very good")
    elif n > 0.1 and n < 0.5:
        humor.append("Good")
    elif n <= 0.1 and n >= -0.1:
        humor.append("Neuter")
    elif n < -0.1 and n > -0.5:
        humor.append("bad")
    else:
        humor.append("Very bad")

humor = pd.DataFrame(humor, columns=['Humor'])
df3 = pd.concat([df2, humor], axis=1)

In [ ]:
df3.head()

In [ ]:
px.histogram(df3['Humor'], title='Humor Distribution')

In [ ]:
px.histogram(df3['HelpfulnessNumerator'], title="Helpfulness Numerator")

In [ ]:
px.scatter(df3, x='Sentimento',
             y='HelpfulnessNumerator',
             color='Humor',
             labels={'HelpfulnessNumerator':'Helpfulness Numerator'})

In [ ]:
px.bar(data_frame=df3.groupby(df3['Humor']).mean(),
         x=df3.groupby(df3['Humor']).mean().index,
         y='HelpfulnessDenominator',
         title="Helpfulness Denominator (Mean)",
         labels={'x':'Humor', 'HelpfulnessDenominator':'Helpfulness Denominator'})